In [1]:
import pandas as pd
from schema import schema

In [2]:
# Read data
df = pd.read_csv(r'data.csv', dtype=schema, encoding='utf-8')

In [3]:
# Pre-processing
df.drop(['state'], axis=1, inplace=True)

In [4]:
df.head()

,iccid,used,plan_c,left,ac_time,product,company,carrier,plan_s,month
0,898607B4071790000000,3.931,10.0,6.069,2018-01-16 15:21:11,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
1,898607B4071790000001,0.000,10.0,10.000,2018-01-16 15:21:44,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
2,898607B4071790000002,4.877,10.0,5.123,2018-01-16 15:19:09,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
3,898607B4071790000003,4.259,10.0,5.741,2018-02-09 11:10:04,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
4,898607B4071790000004,1.740,10.0,8.260,2018-01-16 15:21:28,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2


In [5]:
counts = df['product'].value_counts()

In [6]:
# Filter out non-frequent items
df_flt = df[df['product'].isin(counts[counts >= 100].index)]
# Filter out plan_c == 0 (usage pool)
#df_flt = df_flt[df_flt['plan_c']!=0]

In [7]:
df_flt['product'].value_counts()

SPB132       992065
智能车锁          39578
智能柜           37640
洗衣机           16145
电动摩托车         10729
充电宝            1684
定位器             615
2GWiFi传感器       412
商米POS机          278
Name: product, dtype: int64

In [8]:
# Average usage by device
df_flt.groupby(['product', 'month'])['used'].mean()

product    month
2GWiFi传感器  4         60.042253
           5        266.991063
           6        139.981088
SPB132     2          3.888193
           3          4.418869
           4          4.377335
           5          3.696993
           6          2.676808
充电宝        4          2.956967
           5          2.997328
           6          2.344063
商米POS机     4          0.170667
           5          9.128721
           6          4.017312
定位器        2          2.062519
           3          2.027930
           4          0.912810
           5          1.130042
           6          0.970373
智能柜        2          6.763185
           3          8.455480
           4          7.542408
           5          7.570744
           6          5.472348
智能车锁       2          3.696198
           3          4.710720
           4          4.991057
           5          5.263135
           6          2.820215
洗衣机        2          2.411543
           3          2.983239
           4          

In [9]:
# Split product SPB132
df_noplan = df_flt[df_flt['plan_c']==0]
df_hasplan = df_flt[df_flt['plan_c']!=0]

In [10]:
df_hasplan.head()

,iccid,used,plan_c,left,ac_time,product,company,carrier,plan_s,month
0,898607B4071790000000,3.931,10.0,6.069,2018-01-16 15:21:11,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
1,898607B4071790000001,0.000,10.0,10.000,2018-01-16 15:21:44,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
2,898607B4071790000002,4.877,10.0,5.123,2018-01-16 15:19:09,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
3,898607B4071790000003,4.259,10.0,5.741,2018-02-09 11:10:04,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2
4,898607B4071790000004,1.740,10.0,8.260,2018-01-16 15:21:28,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2


In [11]:
df_hasplan.describe()

,used,plan_c,left,plan_s,month
count,910452.000000,910452.000000,910452.000000,910452.000000,910452.000000
mean,3.999888,11.383774,7.523382,11.178294,4.015080
std,6.693716,5.075498,5.405439,4.709302,1.412036
min,0.000000,10.000000,0.000000,10.000000,2.000000
25%,0.000000,10.000000,4.105000,10.000000,3.000000
50%,4.313000,10.000000,6.114000,10.000000,4.000000
75%,6.131000,10.000000,10.000000,10.000000,5.000000
max,789.941000,30.000000,30.000000,30.000000,6.000000


In [12]:
# Compute paid cost
from metric import get_best_plan_and_cost, compute_cost_with_plan_and_usage
df_hasplan['best_plan'], df_hasplan['best_cost'], df_hasplan['cost_with_plan_c'] = \
    zip(*df_hasplan.apply(lambda row: get_best_plan_and_cost(row['used'], row['plan_c']), axis=1))

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# User perceived cost
_, _, df_hasplan['cost_with_plan_s'] = \
    zip(*df_hasplan.apply(lambda row: get_best_plan_and_cost(row['used'], row['plan_s']), axis=1))

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
df_hasplan.tail()

,iccid,used,plan_c,left,ac_time,product,company,carrier,plan_s,month,best_plan,best_cost,cost_with_plan_c,cost_with_plan_s
1043414,898602B9211770016451,3.304,30.0,26.696,2018-03-08 00:31:57,智能柜,杭州云造科技有限公司,中国移动物联网平台广西分公司,30.0,6,5,2.0,5.0,5.0
1043415,898602B9211770016452,0.000,30.0,30.000,2018-03-08 00:22:20,智能柜,杭州云造科技有限公司,中国移动物联网平台广西分公司,30.0,6,2,1.0,5.0,5.0
1043416,898602B9211770016453,0.000,30.0,30.000,2018-03-08 00:20:46,智能柜,杭州云造科技有限公司,中国移动物联网平台广西分公司,30.0,6,2,1.0,5.0,5.0
1043417,898602B9211770016454,8.103,30.0,21.897,2018-03-01 08:57:26,智能车锁,杭州云造科技有限公司,中国移动物联网平台广西分公司,30.0,6,10,3.0,5.0,5.0
1043418,898602B9211770016459,0.122,30.0,29.878,2018-03-08 00:20:39,智能柜,杭州云造科技有限公司,中国移动物联网平台广西分公司,30.0,6,2,1.0,5.0,5.0


In [15]:
df_hasplan.describe()

,used,plan_c,left,plan_s,month,best_plan,best_cost,cost_with_plan_c,cost_with_plan_s
count,910452.000000,910452.000000,910452.000000,910452.000000,910452.000000,910452.000000,910452.000000,910452.000000,910452.000000
mean,3.999888,11.383774,7.523382,11.178294,4.015080,5.487187,1.988779,3.178831,3.178591
std,6.693716,5.075498,5.405439,4.709302,1.412036,8.444742,0.944541,1.626659,1.702394
min,0.000000,10.000000,0.000000,10.000000,2.000000,2.000000,1.000000,3.000000,3.000000
25%,0.000000,10.000000,4.105000,10.000000,3.000000,2.000000,1.000000,3.000000,3.000000
50%,4.313000,10.000000,6.114000,10.000000,4.000000,5.000000,2.000000,3.000000,3.000000
75%,6.131000,10.000000,10.000000,10.000000,5.000000,10.000000,3.000000,3.000000,3.000000
max,789.941000,30.000000,30.000000,30.000000,6.000000,1000.000000,40.000000,225.382890,229.182890


In [16]:
df_diff_plan = df_hasplan[df_hasplan['plan_c']!=df_hasplan['plan_s']]

In [17]:
len(df_diff_plan)

9354

In [18]:
# Strategy 1: use last month's best plan as next month's plan_c
df_jun = df_hasplan[df_hasplan['month']==6]
df_may = df_hasplan[df_hasplan['month']==5]
# for simplicity, select only devices which exists in both months
iccid_may = df_may.iccid.tolist()
df_jun = df_jun[df_jun['iccid'].isin(iccid_may)]

In [19]:
len(df_jun)

183753

In [20]:
df_jun.describe()

,used,plan_c,left,plan_s,month,best_plan,best_cost,cost_with_plan_c,cost_with_plan_s
count,183753.000000,183753.000000,183753.00000,183753.000000,183753.0,183753.000000,183753.000000,183753.000000,183753.000000
mean,2.683028,11.459133,8.90956,11.253857,6.0,4.491714,1.723931,3.184615,3.175126
std,6.056069,5.201320,5.44715,4.848207,0.0,7.965527,0.952853,1.456205,1.533499
min,0.000000,10.000000,0.00000,10.000000,6.0,2.000000,1.000000,3.000000,3.000000
25%,0.000000,10.000000,4.60000,10.000000,6.0,2.000000,1.000000,3.000000,3.000000
50%,0.049000,10.000000,10.00000,10.000000,6.0,2.000000,1.000000,3.000000,3.000000
75%,5.670000,10.000000,10.00000,10.000000,6.0,5.000000,2.686080,3.000000,3.000000
max,495.010000,30.000000,30.00000,30.000000,6.0,500.000000,24.000000,139.852900,143.652900


In [21]:
# May's best strategy
d = df_may.set_index('iccid')['best_plan'].to_dict()

In [23]:
df_jun['strategy_1'] = df_jun['iccid'].map(d)
df_jun['cost_with_strategy_1'] = df_jun.apply(lambda row: compute_cost_with_plan_and_usage(row['strategy_1'], row['used']), axis=1)

In [24]:
df_jun.head()

,iccid,used,plan_c,left,ac_time,product,company,carrier,plan_s,month,best_plan,best_cost,cost_with_plan_c,cost_with_plan_s,strategy_1,cost_with_strategy_1
859353,898607B4071790000000,5.687,10.0,4.313,2018-01-16 15:21:11,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,6,5,2.703488,3.0,3.0,10,3.0
859354,898607B4071790000001,6.581,10.0,3.419,2018-01-16 15:21:44,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,6,10,3.000000,3.0,3.0,10,3.0
859355,898607B4071790000002,0.000,10.0,10.000,2018-01-16 15:19:09,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,6,2,1.000000,3.0,3.0,2,1.0
859356,898607B4071790000003,0.000,10.0,10.000,2018-02-09 11:10:04,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,6,2,1.000000,3.0,3.0,2,1.0
859357,898607B4071790000004,0.000,10.0,10.000,2018-01-16 15:21:28,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,6,2,1.000000,3.0,3.0,2,1.0


In [25]:
df_jun.describe()

,used,plan_c,left,plan_s,month,best_plan,best_cost,cost_with_plan_c,cost_with_plan_s,strategy_1,cost_with_strategy_1
count,183753.000000,183753.000000,183753.00000,183753.000000,183753.0,183753.000000,183753.000000,183753.000000,183753.000000,183753.000000,183753.000000
mean,2.683028,11.459133,8.90956,11.253857,6.0,4.491714,1.723931,3.184615,3.175126,5.792526,2.017944
std,6.056069,5.201320,5.44715,4.848207,0.0,7.965527,0.952853,1.456205,1.533499,9.537699,1.525269
min,0.000000,10.000000,0.00000,10.000000,6.0,2.000000,1.000000,3.000000,3.000000,2.000000,1.000000
25%,0.000000,10.000000,4.60000,10.000000,6.0,2.000000,1.000000,3.000000,3.000000,2.000000,1.000000
50%,0.049000,10.000000,10.00000,10.000000,6.0,2.000000,1.000000,3.000000,3.000000,5.000000,2.000000
75%,5.670000,10.000000,10.00000,10.000000,6.0,5.000000,2.686080,3.000000,3.000000,10.000000,3.000000
max,495.010000,30.000000,30.00000,30.000000,6.0,500.000000,24.000000,139.852900,143.652900,700.000000,204.565056


In [36]:
(3.184615-1.723931)*183753

268405.06705199997

In [35]:
# where does strategy 1 performs worst?
df_jun['diff_strategy_1'] = df_jun['cost_with_strategy_1'] - df_jun['best_cost']
df_jun.sort_values('diff_strategy_1', ascending=False).drop(['ac_time','left','month','plan_s','carrier'],axis=1).head()

,iccid,used,plan_c,product,company,best_plan,best_cost,cost_with_plan_c,cost_with_plan_s,strategy_1,cost_with_strategy_1,diff_strategy_1
969723,898607B4071790060469,200.794,30.0,SPB132,挚享科技（上海）有限公司,300,16.00000,54.53026,58.33026,2,204.565056,188.565056
970506,898607B4071790061285,198.602,30.0,SPB132,挚享科技（上海）有限公司,300,16.00000,53.89458,57.69458,5,200.248448,184.248448
970745,898607B4071790061528,385.494,30.0,SPB132,挚享科技（上海）有限公司,500,24.00000,108.09326,111.89326,30,108.093260,84.093260
970612,898607B4071790061395,373.395,30.0,SPB132,挚享科技（上海）有限公司,500,24.00000,104.58455,108.38455,30,104.584550,80.584550
969310,898607B4071790060013,302.855,30.0,SPB132,挚享科技（上海）有限公司,300,16.82795,84.12795,87.92795,100,66.827950,50.000000


In [38]:
df_jun.describe()

,used,plan_c,left,plan_s,month,best_plan,best_cost,cost_with_plan_c,cost_with_plan_s,strategy_1,cost_with_strategy_1,diff_strategy_1
count,183753.000000,183753.000000,183753.00000,183753.000000,183753.0,183753.000000,183753.000000,183753.000000,183753.000000,183753.000000,183753.000000,183753.000000
mean,2.683028,11.459133,8.90956,11.253857,6.0,4.491714,1.723931,3.184615,3.175126,5.792526,2.017944,0.294013
std,6.056069,5.201320,5.44715,4.848207,0.0,7.965527,0.952853,1.456205,1.533499,9.537699,1.525269,1.056140
min,0.000000,10.000000,0.00000,10.000000,6.0,2.000000,1.000000,3.000000,3.000000,2.000000,1.000000,0.000000
25%,0.000000,10.000000,4.60000,10.000000,6.0,2.000000,1.000000,3.000000,3.000000,2.000000,1.000000,0.000000
50%,0.049000,10.000000,10.00000,10.000000,6.0,2.000000,1.000000,3.000000,3.000000,5.000000,2.000000,0.000000
75%,5.670000,10.000000,10.00000,10.000000,6.0,5.000000,2.686080,3.000000,3.000000,10.000000,3.000000,0.166336
max,495.010000,30.000000,30.00000,30.000000,6.0,500.000000,24.000000,139.852900,143.652900,700.000000,204.565056,188.565056


In [40]:
(3.184615 - 2.017944)*183753

214379.296263